In [2]:
import numpy as np

In [3]:
def generateDeck(N=8, P=4):
    deck = np.zeros((P*N, 2))
    for i in range(0,P):
        deck[i * N:i * N + N,0] = np.arange(N)
        deck[i * N:i * N + N,1] = i
    return deck